In [111]:
import os
import sys
import glob
import numpy as np
import pandas as pd
from IPython.display import display
import time 
os.chdir('/mnt/BioAdHoc/Groups/vd-ay/hichip-db-loop-calling/')
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)
pd.set_option('max_colwidth', 400)

In [112]:
# setting input and output with jupyter notebook in context 
if 'ipykernel_launcher.py' in sys.argv[0]:
    input_fn = 'results/samplesheets/fastq/2022.03.30.fastq.samplesheet.with_header.tsv'
    output_prefix = 'results/samplesheets/fastq/2022.03.30.fastq.post-check.download_srr_fastqs'
else:
    input_fn = sys.argv[1]
    output_fn = sys.argv[2]

## Helper Functions

In [113]:
def get_log(x, mode='index'):
    
    if mode == 'index':
        logs = glob.glob('results/fastqs/raw/logs/download_srr_fastqs.*-{}'.format(x))
        
    elif mode == 'path':
        info = x.split('/')
        sample_name = info[3]
        srr_id  = info[4].split('.')[0].split('_')[0]
        index = df.loc[(df[0] == sample_name) & (df[3] == srr_id)].iloc[0].name
        logs = glob.glob('results/fastqs/raw/logs/download_srr_fastqs.*-{}'.format(index))
        
    else:
        return('Try again.')
    
    return(logs)

def list_to_str(l, sep=' '):
    l = [str(x) for x in l]
    return(sep.join(l))

In [114]:
def read_log(log):
    
    """
    Read the log and extract status information including
    the sample_name, srr_id, status (as indicated by the end message),
    and the download status (as indicate by the "reads written" message).
    """
    
    status = 0
    download = 1
    sample_name = 'check'
    srr_id = 'check'
    date = ''

    # extract job_id and index
    meta = os.path.basename(log)
    meta = meta.split('.')[1].split('-')
    job_id = meta[0].replace('o', '')
    index = meta[1]

    with open(log, errors='ignore') as fr:
        for line in fr:
            info = line.strip()
            if 'sample_name' in info:
                sample_name = info.split()[1]
            elif 'srr_id' in info:
                srr_id = info.split()[1]
            elif 'Ended: download_srr_fastqs' in info:
                status = 1
            elif 'reads written' in info:
                download = 1
            
    return([index, sample_name, srr_id, job_id, status, download])

In [115]:
def long_to_short_index_list(serial_list):
    
    small_num = serial_list[0]
    serial_list_short = []
    for i in range(1, len(serial_list)):

        prev_num = serial_list[i -1]
        next_num = serial_list[i]

        if (prev_num + 1) != next_num:

            if small_num == prev_num:
                curr_range = '{}'.format(small_num)
                serial_list_short.append(curr_range)
                small_num = next_num

            else:
                curr_range = '{}-{}'.format(small_num, prev_num)
                serial_list_short.append(curr_range)
                small_num = next_num

    if small_num == next_num:
        curr_range = '{}'.format(small_num,)
        serial_list_short.append(curr_range)
    else:
        curr_range = '{}-{}'.format(small_num, next_num)
        serial_list_short.append(curr_range)
        
    return(serial_list_short)

## Check Logs

In [116]:
# load the data
df = pd.read_table(input_fn)
df['sample_index'] = df.index + 1 # add 1 since we'll be using 1-based indexing with sed

In [117]:
# create a dataframe of log information 
logs = glob.glob('results/fastqs/raw/logs/download_srr_fastqs.o*-*')
log_data = []
for log in logs:
    index, sample_name, srr_id, job_id, status, download = read_log(log)
    log_data.append([index, sample_name, srr_id, job_id, status, download, log])
log_df = pd.DataFrame(log_data)
log_df.columns = ['sample_index', 'std_sample_name', 'srr_id', 'job_id', 'end_status', 'read_written_status', 'log']
log_df.drop('sample_index', inplace=True, axis=1)

In [118]:
# merge the log information
df = df.merge(log_df, how='left', on=['std_sample_name', 'srr_id'], indicator='merge')
df.loc[df.end_status.isna(), 'end_status'] = 0
df.loc[df.read_written_status.isna(), 'read_written_status'] = 0
df.end_status = df.end_status.astype(int)
df.read_written_status = df.read_written_status.astype(int)

In [119]:
# get the latest job id for a give sample + srr id combo
df = df.sort_values('job_id', ascending=False).drop_duplicates(['std_sample_name', 'srr_id'])

## Check the presences of SRR FASTQ files

In [120]:
# get the download path
download_tpl = 'results/fastqs/raw/{}/{}_2.fastq.gz'
df.loc[:, 'download_output'] = df.apply(lambda x: download_tpl.format(x[0], x[3]), axis=1)

# find whether the download SRR FASTQ is present
df.loc[:, 'download_present'] = df.loc[:, 'download_output'].apply(os.path.exists).astype(int)

In [121]:
reorder_cols = ['std_sample_name',
                 'gse_id',
                 'gsm_id',
                 'srr_id',
                 'organism',
                 'bio_rep',
                 'tech_rep',
                 'sample_index',
                 'job_id',
                 'end_status',
                 'read_written_status',
                 'download_present',
                 'antibody_target',
                 'restriction_enzyme',
                 'sample_name',
                 'log',
                 'download_output']

In [122]:
df = df[reorder_cols]

In [123]:
header_output = '{}.with_header.tsv'.format(output_prefix)
df.to_csv(header_output, header=True, index=False, sep='\t')

without_header_output = '{}.without_header.tsv'.format(output_prefix)
df.to_csv(without_header_output, header=False, index=False, sep='\t')

## Identifying problem samples

In [124]:
problems = (df.end_status != 1) | (df.read_written_status != 1) | (df.download_present != 1)
problems_df = df.loc[problems,:]
problems_df

,std_sample_name,gse_id,gsm_id,srr_id,organism,bio_rep,tech_rep,sample_index,job_id,end_status,read_written_status,download_present,antibody_target,restriction_enzyme,sample_name,log,download_output
7,mES_100k_cells.GSE101498.Mus_Musculus.H3K27ac.b2,GSE101498,GSM2705038,SRR5831486,Mus_Musculus,2,1,8,NaN,0,0,0,H3K27ac,MboI,mES_100k_cells,NaN,results/fastqs/raw/mES_100k_cells.GSE101498.Mus_Musculus.H3K27ac.b2/SRR5831486_2.fastq.gz
61,HAVIC.GSE154512.Homo_Sapiens.H3K27ac.b1,GSE154512,GSM4672364,SRR12231664,Homo_Sapiens,1,1,62,NaN,0,0,0,H3K27ac,MboI,HAVIC,NaN,results/fastqs/raw/HAVIC.GSE154512.Homo_Sapiens.H3K27ac.b1/SRR12231664_2.fastq.gz


In [125]:
problems_df.loc[:, ['std_sample_name', 'log']]

,std_sample_name,log
7,mES_100k_cells.GSE101498.Mus_Musculus.H3K27ac.b2,NaN
61,HAVIC.GSE154512.Homo_Sapiens.H3K27ac.b1,NaN


## Rerun problematic samples

In [126]:
# convert not run serial IDs to ranges/short format
rerun_indexes = sorted(problems_df.sample_index.tolist())
rerun_indexes = long_to_short_index_list(rerun_indexes)

In [127]:
rerun_indexes

['8', '62']

In [128]:
# generate a string version of not_run
final_rerun = ','.join(rerun_indexes)
final_rerun = 'qsub -t {} workflow/scripts/fastq/download_srr_fastqs.qarray.qsh'.format(final_rerun)

In [129]:
print(final_rerun)

qsub -t 8,62 workflow/scripts/fastq/download_srr_fastqs.qarray.qsh


<p style="color: red"><b>Re-running in progress: 2022.04.02 - 16:30</b></p>